# Lab 3: TableGPT


In this lab, we'll discover the power of code generation models through TableGPT2. The aim is to see how the model can be used in data analysis.

First of all, the notebook is divided into X sections:
0. Installation: This section is dedicated to module installation, model loading and data loading.
1. Guided introduction: Together, we'll discover how to use and evaluate TableGPT2.
2. More questions: You'll need to add at least one new question type to our simple evaluation system.
3. More data sets: You'll need to implement a question with multiple datasets.


IMPORTANT:
- You must work in pairs. You must submit **ONLY ONE NOTEBOOK** for each pair.
- Do not share your work with other pairs.
- You should not use Copilot, ChatGPT or similar tools. At the very least, remove the prompt ...
- <font color='red'>All the things you need to do are indicated in red.</font>


<font color='red'>**FIRST QUESTION:** What are the specificty of the TableGPT2 model?</font> https://huggingface.co/tablegpt/TableGPT2-7B

1. TableGPT2-7B incorporates a unique semantic encoder tailored to interpret tabular data, capturing insights from rows, columns, and entire tables.
2. The model accepts both text and tabular data as input, with tabular data structured as text in the format of a DataFrame's head() result.
3. The model was trained on over 86 billion tokens during continual pretraining and fine-tuned with approximately 2.36 million high-quality query-table-output tuples, encompassing around 593,800 tables.
4. It produces text-based outputs optimized for coding tasks, data interpretation, and business intelligence-focused question answering.
5. While the model emphasizes Chinese corpora, it may have limited support for queries in other languages.

## 0. Setup

In [1]:
!pip install transformers datasets bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    GenerationConfig
)

import pandas as pd
import torch

In [3]:
llm_name = "tablegpt/TableGPT2-7B"

# We want to use 4bit quantization to save memory
quantization_config = BitsAndBytesConfig(
    load_in_8bit=False, load_in_4bit=True
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_name, padding_side="left")
# Prevent some transformers specific issues.
tokenizer.use_default_system_prompt = False
tokenizer.pad_token_id = tokenizer.eos_token_id

# Load LLM.
llm = AutoModelForCausalLM.from_pretrained(
    llm_name,
    quantization_config=quantization_config,
    device_map={"": 0}, # load all the model layers on GPU 0
    torch_dtype=torch.bfloat16, # float precision
)
# Set LLM on eval mode.
llm.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(152064, 3584)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=3584, out_features=3584, bias=True)
          (k_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (v_proj): Linear4bit(in_features=3584, out_features=512, bias=True)
          (o_proj): Linear4bit(in_features=3584, out_features=3584, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (up_proj): Linear4bit(in_features=3584, out_features=18944, bias=False)
          (down_proj): Linear4bit(in_features=18944, out_features=3584, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((3584,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((3584,), eps=1e-0

In [4]:
generation_config = GenerationConfig(
  max_new_tokens=128,
  do_sample=False,
  #do_sample=True,
  # temperature=.7,
  #top_p=.8,
  # top_k=20,
  eos_token_id=tokenizer.eos_token_id,
  pad_token_id=tokenizer.pad_token_id,
)

In [5]:
df = pd.read_csv("hf://datasets/phihung/titanic/train.csv")
df = df.drop("Cabin", axis=1).dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 712 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  712 non-null    int64  
 1   Survived     712 non-null    int64  
 2   Pclass       712 non-null    int64  
 3   Name         712 non-null    object 
 4   Sex          712 non-null    object 
 5   Age          712 non-null    float64
 6   SibSp        712 non-null    int64  
 7   Parch        712 non-null    int64  
 8   Ticket       712 non-null    object 
 9   Fare         712 non-null    float64
 10  Embarked     712 non-null    object 
dtypes: float64(2), int64(5), object(4)
memory usage: 66.8+ KB


## 1.1 Guided Introduction: The Model.

Below there is an example of a prompt that could be used with TableGPT2.

```
Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output".

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5         0       3                            Allen, Mr. William Henry   male 35.0      0      0           373450  8.0500        S
*/

Question: How many child survive? (under 18)
```

The prompt is divided in 3 parts:
1. The global instruction wich is to write python that could answer a question on a specific dataset.
2. The header of the given dataset: 5 first lines of titanic dataset.
3. The question to answer: "How many child survive? (under 18)


First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `generate_answer` function following the comments inside.</font>


In [44]:
example_prompt_template = """Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output", last string in code must consist result in variable output = .

/*
"{var_name}.head(5).to_string(index=False)" as follows:
{df_info}
*/

Question: {user_question}
"""

def generate_answer(prompt, llm=llm, generation_config=generation_config):

  # Create turns with the given prompt.

  # Apply template with the tokenizer. Be careful to return pt tensors on the same device than `llm`.

  # Generate with llm using the given generation config.

  # Decode and select the answer to return.

    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = inputs.to(llm.device)

    outputs = llm.generate(
        inputs["input_ids"],
        max_new_tokens=generation_config.max_new_tokens,
        do_sample=generation_config.do_sample,
        eos_token_id=generation_config.eos_token_id,
        pad_token_id=generation_config.pad_token_id,
    )

    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    start_index = decoded_output.find("output =")
    if start_index != -1:
        return decoded_output[start_index:]  # Return only the Python code starting from "output ="
    return decoded_output

prompt = example_prompt_template.format(
    var_name="df",
    df_info=df.head(5).to_string(index=False),
    user_question="How many child survive? (under 18)",
)

answer = generate_answer(prompt)

print(prompt)
print("\n*****\n")
print(answer)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


Given access to several pandas dataframes, write the Python code to answer the user's question.
The answer should be store in a variable named "output", last string in code must consist result in variable output = .

/*
"df.head(5).to_string(index=False)" as follows:
 PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked
           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S
           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C
           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S
           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S
           5     

In [45]:
answer

'output = .\n\n/*\n"df.head(5).to_string(index=False)" as follows:\n PassengerId  Survived  Pclass                                                Name    Sex  Age  SibSp  Parch           Ticket    Fare Embarked\n           1         0       3                             Braund, Mr. Owen Harris   male 22.0      1      0        A/5 21171  7.2500        S\n           2         1       1 Cumings, Mrs. John Bradley (Florence Briggs Thayer) female 38.0      1      0         PC 17599 71.2833        C\n           3         1       3                              Heikkinen, Miss. Laina female 26.0      0      0 STON/O2. 3101282  7.9250        S\n           4         1       1        Futrelle, Mrs. Jacques Heath (Lily May Peel) female 35.0      1      0           113803 53.1000        S\n           5         0       3                            Allen, Mr. William Henry   male 35.0      0      0           373450  8.0500        S\n*/\n\nQuestion: How many child survive? (under 18)\nTo determine the

## 1.2 Guided Introduction: The Answer.

As you can see, the model answer with some generated code.

```
Python code:
```python
# Filter the dataframe to include only passengers under the age of 18
children = df[df['Age'] < 18]

# Count the number of children who survived
child_survivors = children[children['Survived'] == 1]

# Save the answer in the variable output
output = len(child_survivors)
```

So we will need to execute it, but there is some difficulty:
1. Sometime, the llm answer with \`\`\`python ... \`\`\`, sometime the llm answer directly with the code. We need to handle both cases.
2. We need to recover the variable output from the execution.
3. We need to evaluate single value and list of values.


First, we will implement a function that generate an answer for this prompt.

<font color='red'>TODO: Fill in the `exec_answer` function following the comments inside.</font>


In [29]:
re.findall(r"```(?:python)?\n(.*?)```", answer, flags=re.DOTALL)[0].strip()

"# Filter the dataframe where Survived is 0\nfiltered_df = df[df['Survived'] == 0]\n\n# Get the unique values in Embarked for the filtered dataframe\nunique_embarked_values = filtered_df['Embarked'].unique()\n\n# Calculate the length of the unique values\nlen_unique_embarked_values = len(unique_embarked_values)\n\n# Save the answer in the output variable\noutput = len_unique_embarked_values\n\n# Print the output\nprint(output)"

In [46]:
import re

def exec_answer(answer, gold):
    code = re.findall(r"```(?:python)?\n(.*?)```", answer, flags=re.DOTALL)[0].strip()

    local_vars = {"df": df}  #
    try:
        exec(code, {}, local_vars)
    except Exception as e:
        print(f"Error during execution: {e}")
        return False

    if "output" not in local_vars:
        print("Error: 'output' variable not found in the executed code.")
        return False

    output = local_vars["output"]
    try:
        return output == gold
    except Exception as e:
        print(f"Error during comparison: {e}")
        return False

  # Extract the code from the answer. Be careful, the code is now always in ``` ```.

  # Execute the code, https://docs.python.org/3/library/functions.html#exec
  # if the code work: Return True or False based on output == gold (be careful to handle iterable !)
  # if the code don't work return False.


print(exec_answer(answer, 61))

True


## 1.3 Guided Introduction: The Question.

Now we want to automatically generate questions to evaluate the performance of our model. There are benchmarks on this subject, but here we want to practice code by generating the questions ourselves.

We will generate some basic filter questions.

<font color='red'>TODO: Fill in the `generate_filter_question` function following the comments inside.</font>


In [39]:
import random

def generate_random_question(generate_function, df, k=1, seed=42):
  random.seed(seed)
  return [generate_function(df) for _ in range(k)]

def generate_filter_question(df):

  # Get a random target column and a random filter column (be careful they should be differnts)
  columns = df.columns.tolist()
  target_column = random.choice(columns)
  filter_column = random.choice([col for col in columns if col != target_column])
  filter_values = df[filter_column].dropna().unique()
  filter_value = random.choice(filter_values)

  # Get a random filter value inside the filer column. Avoid NaN values.
  filtered_df = df[df[filter_column] == filter_value]

  # Compute the correct answer for the given target column, filter column and filter value.
  correct_answer = filtered_df[target_column].nunique()

  question_template = "Calculate the number of unique values in the column {target_column} where the column {filter_column} has the value {filter_value}. Store the result in a variable named output."
  question = question_template.format(
      target_column=target_column,
      dataframe=df,
      filter_column=filter_column,
      filter_value=filter_value
  )
  return {"question": question, "answer": correct_answer}
  # return formated question and associated answer in a dict {"question":[question], "answer":[answer]}

generate_random_question(generate_filter_question, df, k=5)

[{'question': 'Calculate the number of unique values in the column Embarked where the column Survived has the value 0. Store the result in a variable named output.',
  'answer': 3},
 {'question': 'Calculate the number of unique values in the column Sex where the column Name has the value Stankovic, Mr. Ivan. Store the result in a variable named output.',
  'answer': 1},
 {'question': 'Calculate the number of unique values in the column Pclass where the column Survived has the value 0. Store the result in a variable named output.',
  'answer': 3},
 {'question': 'Calculate the number of unique values in the column Fare where the column SibSp has the value 1. Store the result in a variable named output.',
  'answer': 94},
 {'question': 'Calculate the number of unique values in the column PassengerId where the column Pclass has the value 3. Store the result in a variable named output.',
  'answer': 355}]

## 1.4 Guided Introduction: The Evaluation.

The last step in this section is to evaluate our model on 20 random questions! We'll use simple accuracy.

You should have an accuracy between 0.9 and 1.

<font color='red'>TODO: Follow instruction in comment of the cell below.</font>

<font color='green'>BONUS: Investigate on errors and improve our prompt/parsing to solve them.</font>


In [43]:
from tqdm import tqdm

# Generate 20 random question
dict_q_ans = generate_random_question(generate_filter_question, df, k=20)
estimation = []
for i in tqdm(range(len(dict_q_ans))):
  prompt = example_prompt_template.format(
      var_name="df",
      df_info=df.head(5).to_string(index=False),
      user_question=dict_q_ans[i]["question"],
  )
  answer = generate_answer(prompt)
  estimation.append(exec_answer(answer, dict_q_ans[i]["answer"]))

# Iterate over question to format prompt, generate answer and execute answer.
# Report the Accuracy
acc = sum(estimation) / len(estimation)
print("Acc: ", acc)

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  5%|▌         | 1/20 [00:11<03:40, 11.58s/it]

3


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 10%|█         | 2/20 [00:23<03:28, 11.57s/it]

1


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 15%|█▌        | 3/20 [00:34<03:15, 11.49s/it]

...

1


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 95%|█████████▌| 19/20 [03:40<00:11, 11.65s/it]

1


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
100%|██████████| 20/20 [03:51<00:00, 11.59s/it]

259
Acc:  1.0


Accuracy: 1.0

## 2. More Questions.

Now it's your turn to imagine a type of question ("How many ..."). Implement a function to generate new type of question. Verify that our previous code work with your new question then evaluate it.

<font color='red'>TODO: Generate **AT LEAST ONE** new type of question and report this new question accuracy.</font>


In [49]:
def generate_count_question(df):

  df = df.dropna()
  columns = df.columns.tolist()
  filter_column = random.choice(columns)
  filter_values = df[filter_column].unique()
  filter_value = random.choice(filter_values)

  # Get a random filter value inside the filer column. Avoid NaN values.
  filtered_df = df[df[filter_column] == filter_value]

  # Compute the correct answer for the given target column, filter column and filter value.
  correct_answer = filtered_df.shape[0]
  question_template = "How many rows in the DataFrame have no missing values and where the value in {filter_column} is {filter_value}? Store the result in a variable named output."
  question = question_template.format(
      filter_column=filter_column,
      filter_value=filter_value
  )
  return {"question": question, "answer": correct_answer}

generate_random_question(generate_count_question, df, k=5)

[{'question': 'How many rows in the DataFrame have no missing values and where the value in Embarked is S? Store the result in a variable named output.',
  'answer': 554},
 {'question': 'How many rows in the DataFrame have no missing values and where the value in PassengerId is 351? Store the result in a variable named output.',
  'answer': 1},
 {'question': 'How many rows in the DataFrame have no missing values and where the value in Name is Stankovic, Mr. Ivan? Store the result in a variable named output.',
  'answer': 1},
 {'question': 'How many rows in the DataFrame have no missing values and where the value in Pclass is 2? Store the result in a variable named output.',
  'answer': 173},
 {'question': 'How many rows in the DataFrame have no missing values and where the value in Survived is 0? Store the result in a variable named output.',
  'answer': 424}]

In [50]:
# Generate 20 random question
dict_q_ans = generate_random_question(generate_count_question, df, k=20)
estimation = []
for i in tqdm(range(len(dict_q_ans))):
  prompt = example_prompt_template.format(
      var_name="df",
      df_info=df.head(5).to_string(index=False),
      user_question=dict_q_ans[i]["question"],
  )
  answer = generate_answer(prompt)
  estimation.append(exec_answer(answer, dict_q_ans[i]["answer"]))

# Iterate over question to format prompt, generate answer and execute answer.
# Report the Accuracy
acc = sum(estimation) / len(estimation)
print("Acc: ", acc)

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  5%|▌         | 1/20 [00:10<03:27, 10.91s/it]/usr/local/lib/py

173


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 25%|██▌       | 5/20 [00:54<02:44, 10.96s/it]/usr/local/lib/python3.11/dist-packages/transformers/ge

1


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 50%|█████     | 10/20 [01:52<01:54, 11.48s/it]/usr/local/lib/python3.11/dist-packages/transformers/g

...

1


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 75%|███████▌  | 15/20 [02:50<00:58, 11.67s/it]/usr/local/lib/python3.11/dist-packages/transformers/g

712


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 90%|█████████ | 18/20 [03:24<00:23, 11.55s/it]/usr/local/lib/python3.11/dist-packages/transformers/g

Acc:  0.9


Accuracy: 0.9

## 3. More datasets.

Below we load a new dataset: "adult_income_dataset".

<font color='red'>TODO: Evaluate our questions on this new dataset. Report the accuracy. Comment Any differences.</font>

<font color='green'>BONUS: Try to find a prompt that answer this question: What is the mean salary of titanic surviror based on adult dataset.</font>

In [51]:
adult = pd.read_csv("hf://datasets/meghana/adult_income_dataset/adult.csv")
adult.info()

titanic = df


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [55]:
df = adult

In [56]:
# Generate 20 random question
dict_q_ans = generate_random_question(generate_filter_question, df, k=20)
estimation = []
for i in tqdm(range(len(dict_q_ans))):
  prompt = example_prompt_template.format(
      var_name="df",
      df_info=df.head(5).to_string(index=False),
      user_question=dict_q_ans[i]["question"],
  )
  answer = generate_answer(prompt)
  estimation.append(exec_answer(answer, dict_q_ans[i]["answer"]))

# Iterate over question to format prompt, generate answer and execute answer.
# Report the Accuracy
acc = sum(estimation) / len(estimation)
print("Acc: ", acc)

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  5%|▌         | 1/20 [00:14<04:43, 14.91s/it]

114


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 10%|█         | 2/20 [00:27<04:07, 13.73s/it]

63


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 15%|█▌        | 3/20 [00:39<03:36, 12.74s/it]

...

2


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 95%|█████████▌| 19/20 [04:03<00:12, 12.23s/it]

22


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
100%|██████████| 20/20 [04:15<00:00, 12.79s/it]

9
Acc:  1.0


In [57]:
# Generate 20 random question
dict_q_ans = generate_random_question(generate_count_question, df, k=20)
estimation = []
for i in tqdm(range(len(dict_q_ans))):
  prompt = example_prompt_template.format(
      var_name="df",
      df_info=df.head(5).to_string(index=False),
      user_question=dict_q_ans[i]["question"],
  )
  answer = generate_answer(prompt)
  estimation.append(exec_answer(answer, dict_q_ans[i]["answer"]))

# Iterate over question to format prompt, generate answer and execute answer.
# Report the Accuracy
acc = sum(estimation) / len(estimation)
print("Acc: ", acc)

  0%|          | 0/20 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
  5%|▌         | 1/20 [00:12<03:53, 12.28s/it]/usr/local/lib/py

8025


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 20%|██        | 4/20 [00:48<03:13, 12.11s/it]/usr/local/lib/python3.11/dist-packages/transformers/ge

10


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 30%|███       | 6/20 [01:13<02:55, 12.54s/it]

...

1


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 90%|█████████ | 18/20 [03:35<00:23, 11.84s/it]

48842


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.8` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:650: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `20` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
 95%|█████████▌| 19/20 [03:47<00:11, 11.84s/it]/usr/local/lib/python3.11/dist-packages/transformers/g

95
Acc:  0.9


In [58]:
dict_q_ans

[{'question': 'How many rows in the DataFrame have no missing values and where the value in capital-gain is 1055? Store the result in a variable named output.',
  'answer': 37},
 {'question': 'How many rows in the DataFrame have no missing values and where the value in age is 41? Store the result in a variable named output.',
  'answer': 1235},
 {'question': 'How many rows in the DataFrame have no missing values and where the value in education is Bachelors? Store the result in a variable named output.',
  'answer': 8025},
 {'question': 'How many rows in the DataFrame have no missing values and where the value in fnlwgt is 173271? Store the result in a variable named output.',
  'answer': 1},
 {'question': 'How many rows in the DataFrame have no missing values and where the value in workclass is Never-worked? Store the result in a variable named output.',
  'answer': 10},
 {'question': 'How many rows in the DataFrame have no missing values and where the value in workclass is Self-emp-i

Based on the results, we see that the model copes with the task perfectly, which means that the questions were formulated quite accurately.